In [ ]:
!pip uninstall kaggle -y
!pip uninstall --upgrade.pip
!pip install kaggle==1.5.6
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

Found existing installation: kaggle 1.5.12
Uninstalling kaggle-1.5.12:
  Successfully uninstalled kaggle-1.5.12

Usage:   
  pip3 uninstall [options] <package> ...
  pip3 uninstall [options] -r <requirements file> ...

no such option: --upgrade.pip
     |████████████████████████████████| 58 kB 5.9 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72857 sha256=a65beeb4767e6b0cccab25dbeb45514fe7f81a601857fee2ac497e0ced42430c
  Stored in directory: /root/.cache/pip/wheels/aa/e7/e7/eb3c3d514c33294d77ddd5a856bdd58dc9c1fabbed59a02a2b
Successfully built kaggle


In [ ]:
!kaggle competitions download -c 2021-ai-w3-p1

  0% 0.00/26.3k [00:00<?, ?B/s]
100% 26.3k/26.3k [00:00<00:00, 20.5MB/s]


In [ ]:
!ls

2021-ai-w3-p1.zip  drive  sample_data


In [ ]:
!unzip 2021-ai-w3-p1.zip

Archive:  2021-ai-w3-p1.zip
  inflating: submit_sample.csv       
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
#필요한 라이브러리 선언
import numpy as np
import pandas as pd
import torch
import torch.optim as optim

#seed고정
torch.manual_seed(1)

In [ ]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
submit=pd.read_csv("submit_sample.csv")

In [ ]:
train["year"]=train["year"]//10000-2000
test["year"]=test["year"]//10000-2000

In [ ]:
x=train.drop(["avgPrice"],axis=1)
y=train["avgPrice"]

In [ ]:
x=np.array(x)
x=torch.FloatTensor(x)
y=torch.LongTensor(y)
test=np.array(test)
test=torch.FloatTensor(test)

In [ ]:
# 모델 초기화
W = torch.zeros((5, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.Adam([W, b], lr=0.5)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    # Matrix 연산!!
    hypothesis = x.matmul(W) + b # or .mm or @

    # cost 계산
    cost = torch.mean((hypothesis - y) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch%100==0:
      print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
          epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()
      ))

Epoch    0/1000 hypothesis: tensor([0., 0., 0.,  ..., 0., 0., 0.]) Cost: 12013343.000000
Epoch  100/1000 hypothesis: tensor([-10.9614, 408.5748, 198.3862,  ..., 603.1245, 612.4240, 934.1920]) Cost: 4310192.500000
Epoch  200/1000 hypothesis: tensor([ 444.6765,  821.4658,  597.1349,  ..., 1122.8925, 1203.9144,
        1494.6544]) Cost: 3394182.750000
Epoch  300/1000 hypothesis: tensor([ 965.3365, 1231.7627, 1018.7198,  ..., 1617.5792, 1776.2203,
        1978.2260]) Cost: 2790558.250000
Epoch  400/1000 hypothesis: tensor([1429.6974, 1596.5800, 1391.9766,  ..., 2051.6047, 2277.4404,
        2395.4487]) Cost: 2402104.750000
Epoch  500/1000 hypothesis: tensor([1810.8715, 1895.8475, 1697.5223,  ..., 2405.5713, 2685.4290,
        2732.6646]) Cost: 2177013.500000
Epoch  600/1000 hypothesis: tensor([2103.3730, 2125.6245, 1932.0153,  ..., 2677.0537, 2997.6438,
        2990.1328]) Cost: 2059683.375000
Epoch  700/1000 hypothesis: tensor([2314.4692, 2291.7107, 2101.6111,  ..., 2873.6443, 3222.9741,


In [ ]:
pred = test.matmul(W) + b
for i in range(len(pred)):
  submit["Expected"][i]=pred[i].item()
submit.to_csv("submit.csv",mode='w',index=False)